In [1]:
import chessdotcom
import datetime
import plotly.express as px
import pandas as pd

In [2]:
USERNAMES = ['<your_username>', '<your_username>', '<your_username>']
START_YEAR = 2014
END_YEAR = 2020
MOVING_AVERAGE_WINDOW_GAMES = 1000

In [3]:
def get_games(usernames, start_year: int, end_year: int):
    games = []
    for username in usernames:
        for year in range(start_year, end_year + 1):
            for month in range(1, 13):
                monthly_games = chessdotcom.caller.get_player_games_by_month(username, year=year, month=month)
                games.extend(monthly_games.json['games'])
                # print(f"Games for Year: {year}, Month: {month}, User: {username} retrieved")
    return games

def game_dict(game: dict, usernames: str, time_format='blitz', rules='chess'):
    if game['white']['username'] in usernames:
        rating = game['white']['rating']
    else:
        rating = game['black']['rating']
    
    if game['time_class'] != time_format or game['rules'] != rules:
        return {}
    
    game_dict = {
        'time': pd.to_datetime(game['end_time'], unit='s'),
        'rating': rating,
        'format': game['time_class']
    }
    return game_dict

def plot_games(game_df: pd.DataFrame):
    fig = px.line(game_df, x="time", y="Moving Average", title='Chess Rating Over Time')
    fig.show()

def create_game_plot():
    games = get_games(USERNAMES, START_YEAR, END_YEAR)
    game_df = pd.DataFrame([game_dict(game, USERNAMES) for game in games if game_dict(game, USERNAMES)])
    game_df.sort_values(by='time', inplace=True)
    game_df['Moving Average'] = game_df.rolling(window=MOVING_AVERAGE_WINDOW_GAMES).mean()
    plot_games(game_df)


In [5]:
create_game_plot()